In [1]:
import tensorflow as tf
print(tf.__version__)

import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense,Input,Embedding,Bidirectional,LSTM,Lambda,Reshape
from tensorflow.keras.models import Model

2.3.1


In [2]:
import os

os.path.join('content/','train')

'content/train'

In [8]:
df=pd.read_csv("adult_video_cap_detection/final_data.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'adult_video_cap_detection/final_data.csv'

In [9]:
df=df[['titles','label']]
df.head()

NameError: name 'df' is not defined

In [5]:
# cleaning data

import re

class CreateDataset():
  def __init__(self,df):
    self.df=df

  def __call__(self):
    train_data=[]
    values,labels=self.df['titles'].values,self.df['label'].values
    for value in values:
      x=re.sub("[^0-9a-zA-Z]+"," ",str(value))
      train_data.append(x)

    return train_data,labels


In [6]:
dir(CreateDataset)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [7]:
data=CreateDataset(df)

In [8]:
x_train_data,y_train_data=data()

In [9]:
for i,j in zip(x_train_data,y_train_data):
  print(f'{i}----{j}')

S DAY I SHALL TRY amp STAY POSITIVE BUT I HAVE A LOT ON MY PLATE 2DAY I BARELY EVEN HAVE TIME 2 TWIT NO FAIR ----0
Guy with 2 teens ends in ass fuck----1
looking for friends im new to this twitter ----0
not feeling v good abt myself ----0
AroundTheHorn That championship bout was pretty tactical but in a BORING way Leites shoulda came out blazing in the 5th Depressed ----0
VID 20161217 WA0086----1
Amateur Porn Show 3128646 2 amature video----1
bad luck follows me around closer than my own shadow ----0
JPBrancati omg I hear ya It s horrid ----0
I want to go out and jog right now but my parents would tell me it s too late HOLY CRAP I MISS MY BABY WAAAAAAAAAAAAAAY TOO MUCH ----0
 y----1
tweetdeck isn t working for me i can t keep up with watkins on here ----0
He deceived me that he is taking me to his house but ended up fucking in the local bush----1
epic amateur cumshot compilation----1
Hottie strips bikini----1
English----1
knee still not ready good luck to my squad pacrim all stars----0

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
# testing tf.pad
# t = tf.constant([[1, 2, 3], [4, 5, 6]])
# paddings = tf.constant([[1, 1,], [2, 2]])

# tf.pad(t, paddings, "CONSTANT")

In [12]:
tokenizer=Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(x_train_data)
sequences=tokenizer.texts_to_sequences(x_train_data)


In [13]:
vocab=tokenizer.word_index

In [14]:
sequences[:2]

[[21, 338, 48, 2, 156, 30, 53, 3, 124],
 [15563, 424, 2678, 51, 207, 983, 14, 3352, 5098, 690, 3, 3149]]

In [15]:
max([len(i) for i in sequences])

52

In [16]:
train_sequence=pad_sequences(sequences,maxlen=60)
train_sequences=tf.convert_to_tensor(train_sequence)

In [17]:
train_sequences

<tf.Tensor: shape=(43599, 60), dtype=int32, numpy=
array([[   0,    0,    0, ...,   53,    3,  124],
       [   0,    0,    0, ...,  690,    3, 3149],
       [   0,    0,    0, ..., 6627,   16,  428],
       ...,
       [   0,    0,    0, ...,  193,  470, 2889],
       [   0,    0,    0, ...,  994,  535,  191],
       [   0,    0,    0, ..., 5292,  415,   57]])>

In [18]:
y_train_label=tf.one_hot(y_train_data,2)

In [19]:
y_train_label

<tf.Tensor: shape=(43599, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)>

In [20]:
class New_Model(tf.keras.Model):
    def __init__(self,units=512):
        super(New_Model,self).__init__()
        self.embedding_layer=tf.keras.layers.Embedding(len(vocab),100)
        self.lstm_layer=tf.keras.layers.LSTM(units,return_sequences=True)
        self.dense_layer_1=tf.keras.layers.Dense(units/2,activation='relu')
        self.dense_layer_2=tf.keras.layers.Dense(2,activation='softmax')
        self.dropout_layer=tf.keras.layers.Dropout(0.2)
        self.flat_layer=tf.keras.layers.Flatten()

    def call(self,inputs):
        x=self.embedding_layer(inputs)
        x=self.lstm_layer(x)
        x=self.flat_layer(x)
        x=self.dropout_layer(x)
        x=self.dense_layer_1(x)
        x=self.dense_layer_2(x)

        return x




In [21]:
model=New_Model()

In [22]:
x_in = tf.keras.layers.Input(shape=(60,), dtype=tf.float32)
initializer=model(x_in, training=False)

In [23]:
train_sequences.shape,y_train_label.shape

(TensorShape([43599, 60]), TensorShape([43599, 2]))

In [24]:
train_sequences[1],y_train_label[1]

(<tf.Tensor: shape=(60,), dtype=int32, numpy=
 array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 15563,   424,  2678,    51,   207,   983,
           14,  3352,  5098,   690,     3,  3149])>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 0.], dtype=float32)>)

In [25]:
AUTO=tf.data.experimental.AUTOTUNE
train_data=tf.data.Dataset.from_tensor_slices((train_sequences[:int(0.8*len(train_sequences))],y_train_label[:int(0.8*len(train_sequences)]))).batch(32,drop_remainder=True).prefetch(AUTO)
valid_data=tf.data.Dataset.from_tensor_slices((train_sequences[int(0.8*len(train_sequences)):],y_train_label[int(0.8*len(train_sequences):]))).batch(32,drop_remainder=True).prefetch(AUTO)

In [26]:
train_data,valid_data

<PrefetchDataset shapes: ((32, 60), (32, 2)), types: (tf.int32, tf.float32)>

In [27]:
type(data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [32]:
filepath='porn_detection.h5'
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=filepath,monitor='loss', save_best_only=True,verbose=1,save_weights_only=True,mode='auto',period=1)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])  # we may use SGD or mini-gradient-descent 

In [29]:
model.fit(data,steps_per_epoch=int(len(train_sequences)/32),validation_data=valid_data,epochs=5,verbose=1)

Epoch 1/5
1362/1362 [==============================] - 901s 661ms/step - loss: 0.0717 - accuracy: 0.9744
Epoch 2/5
1362/1362 [==============================] - 1009s 741ms/step - loss: 0.0147 - accuracy: 0.9956
Epoch 3/5
1362/1362 [==============================] - 671s 493ms/step - loss: 0.0094 - accuracy: 0.9971
Epoch 4/5
1362/1362 [==============================] - 673s 494ms/step - loss: 0.0052 - accuracy: 0.9985
Epoch 5/5
1362/1362 [==============================] - 661s 485ms/step - loss: 0.0045 - accuracy: 0.9989


In [30]:
model.summary()

Model: "new__model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 100)           3975000   
_________________________________________________________________
lstm (LSTM)                  (None, 60, 512)           1255424   
_________________________________________________________________
dense (Dense)                (None, 256)               7864576   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
_________________________________________________________________
dropout (Dropout)            (None, 30720)             0         
_________________________________________________________________
flatten (Flatten)            (None, 30720)             0         
Total params: 13,095,514
Trainable params: 13,095,514
Non-trainable params: 0
____________________________________________

In [ ]:
valid_data